Rename a bunch of files using Git and renamer?



In [3]:
var execSync = require('child_process').execSync;
try {
    require.resolve('renamer');
} catch (e) {
    execSync('npm install renamer');
}
var renamer = require('renamer');
var renameUsingGit = (root, match, find, replace) => {
    var files = renamer.expand(path.join(root, match));
    var results = renamer.replace({
        files: files.filesAndDirs, 
        find: find, 
        replace: replace});
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git ' 
                 + '--work-tree=' + JSON.stringify(root)
                 + ' --git-dir=' + JSON.stringify(path.join(root, '.git'))
                 + ' mv ' + JSON.stringify(r.before) 
                 + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
(renameUsingGit);


In [4]:
// test it
// rename all scss files to less
renameUsingGit(
    '/Users/briancullinan/Documents/portal',
    'src/**/*.scss',
    '.scss',
    '.less');

How to use memory-fs and rewire to audit cli events?



In [1]:
var path = require('path');
var execSync = require('child_process').execSync;
try {
    require.resolve('mock-require');
    require.resolve('typescript')
} catch (e) {
    execSync('npm install mock-require typescript');
}
// TODO: use https://github.com/jhnns/rewire for replacing variables, something like https://github.com/mariocasciaro/object-path?
var mock = require('mock-require');

var mockTypescriptFs = (memfs) => {
    var fs = require('fs');
    var fsMock = {};
    Object.assign(fsMock, fs);
    var prototype = Object.getPrototypeOf(memfs);
    Object.keys(prototype)
        .forEach(k => {
            var orig = prototype[k].bind(memfs);
            fsMock[k] = (a1, a2, a3, a4, a5) => {
                try {
                    return orig(a1, a2, a3, a4, a5);
                } catch (e) {
                    return fs[k](a1, a2, a3, a4, a5);
                }
            }
        });
    
    ['writeFileSync', 'writeFile']
        .forEach(k => {
            var orig = fsMock[k];
            var errorFunc = (e, a1, a2, a3, a4) => {
                if(e && e.code == 'ENOENT') {
                    try {
                        fsMock['mkdirpSync'](path.dirname(a1));
                        return orig(a1, a2, a3, a4);
                    } catch (e2) {
                        throw e; // return original error
                    }
                }
                if(typeof a4 == 'function') {
                    return a4(e);
                } else if(typeof a3 == 'function') {
                    return a3(e);
                }
            };
            fsMock[k] = (a1, a2, a3, a4) => {
                if(typeof a4 == 'function') {
                    var cb = a4;
                    a4 = (e) => {
                        return errorFunc(e, a1, a2, a3, cb);
                    };
                } else if(typeof a3 == 'function') {
                    var cb = a3;
                    a3 = (e) => {
                        return errorFunc(e, a1, a2, cb);
                    };
                }
                try {
                    return orig(a1, a2, a3, a4);
                } catch (e) {
                    return errorFunc(e, a1, a2, a3, a4);
                }
            }
        });
    
    ['existsSync']
        .forEach(k => {
            var orig = fsMock[k];
            fsMock[k] = (a1, a2, a3, a4) => {
                return orig(a1, a2, a3, a4)
                    || fs[k](a1, a2, a3, a4);
            }
        });
    
    ["stat", "readdir", "mkdirp", "rmdir", "unlink", "readlink"]
        .forEach(k => {
            var orig = fsMock[k];
            fsMock[k] = (path, cb) => {
                orig(path, (err, result) => {
                    if(err) {
                        return fs[k](path, cb);
                    }
                    if(typeof cb == 'function') {
                        return cb(err, result);
                    }
                })
            };
        });
    
    ["exists"]
        .forEach(k => {
            var orig = fsMock[k];
            fsMock[k] = (path, cb) => {
                orig(path, (result) => {
                    if(!result) {
                        return fs[k](path, cb);
                    }
                    if(typeof cb == 'function') {
                        return cb(result);
                    }
                });
            };
        });
    
    ["mkdir", "readFile"]
        .forEach(k => {
            var orig = fsMock[k];
            fsMock[k] = (path, opts, cb) => {
                orig(path, opts, (err, result) => {
                    if(k == 'readFile') {
                        return fs[k](path, opts, cb);
                    }
                    if(typeof opts == 'function') {
                        return opts(err, result);
                    } else if(typeof cb == 'function') {
                        return cb(err, result);
                    }
                });
            };
        });
    
    mock('fs', fsMock);

    var ts = require('typescript');
    var tsMock = {};
    Object.assign(tsMock, ts);
    var toMock = {
        readFile: 'readFileSync',
        //writeFile: 'writeFileSync',
        //fileExists: 'existsSync',
        //directoryExists: 'existsSync',
        //createDirectory: 'mkdirSync',
        //getDirectories: 'readdirSync'
    };
    Object.keys(ts.sys)
        .forEach(k => {
            var orig = tsMock.sys[k];
            tsMock.sys[k] = (a1, a2, a3, a4, a5) => {
                if(typeof toMock[k] !== 'undefined'
                    && typeof fsMock[toMock[k]] !== 'undefined') {
                    try {
                        return fsMock[toMock[k]](a1, a2, a3, a4, a5);
                    } catch (e) {
                        // typescript.sys returns undefined on error
                        return;
                    }
                }
                return orig(a1, a2, a3, a4, a5);
            };
        });
    mock('typescript', tsMock);
    return mock;
};
(mockTypescriptFs);

// TODO: search github for projects like graceful-fs and then search google for projects that use graceful-fs and see if we can record some filesystem activity



ReferenceError: mountAndRewireAngular is not defined